In [4]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
import seaborn as sns
import sklearn.metrics as sm

base_path = '/home/maj/data/insideairbnb/processed/'
train = pd.read_csv(base_path + 'train.csv', low_memory=False)
validate = pd.read_csv(base_path + 'validate.csv', low_memory=False)
test = pd.read_csv(base_path + 'test.csv', low_memory=False)
train_validate = pd.concat([train, validate], keys=['train', 'val'])
drop_columns = ['price_log', 'host_has_profile_pic', 'is_business_travel_ready', 'distance_to_san_diego_zoo']
X_train = train.drop(drop_columns, axis=1).to_numpy()
X_validate = validate.drop(drop_columns, axis=1).to_numpy()
X_train_validate = train_validate.drop(drop_columns, axis=1).to_numpy()
X_test = test.drop(drop_columns, axis=1).to_numpy()
# SVR with {'C': 4, 'epsilon': 0.05, 'gamma': 0.0078125} (= 2**-7)
m_svr = SVR(kernel='rbf', C=4, epsilon=0.05, gamma=2**-7).fit(X_train_validate, train_validate.price_log.to_numpy())
print("SVR R^2: %f" % m_svr.score(X_test, test.price_log.to_numpy()))
print("SVR MSE: %f" % sm.mean_squared_error(test.price_log.to_numpy(), m_svr.predict(X_test).flatten()))
ax = sns.scatterplot(test.price_log.to_numpy(), m_svr.predict(X_test).flatten())

0.724244947573807

In [ ]:
# KNNR with {'n_neighbors': 7, 'p': 1}
m_knnr KNeighborsRegressor(algorithm='brute'),
                    param_grid=dict(n_neighbors=list(range(1,40)),
                                    p=[1, 2]),
                    cv=[(train_validate.index.get_locs(['train']), train_validate.index.get_locs(['val']))],
                    n_jobs=20).fit(X_train_validate, train_validate.price_log.to_numpy())
            

In [ ]:
# DNN with {'neurons': 64}